# Agent Based Staircase Model

## Using the Mesa Library, this notebook implements an agent-based approach to modeling the travel of people up and down a staircase simultaneously. 

Each agent has the following characteristics:
- Age (years)
- Sex (M/F)
- Height (cm)
- Weight (kg)
- Stepping Pattern (traditional: one footstep per stair, double: two footsteps per stair, skip: only step on every other stair)
    - Sampled from a distribution dependent on the age, height, and weight of the agent
- Intended Final Destination
- Desired Velocity $v$ (m/s)
- Ascending/Descending

The following parameters are used to define the staircase domain:
- The number of stairs $N$
- The length of each step ${l}_{i = 1}^N $
- The width of each step ${w}_{i=1}^N $

It is assumed that the steps are continuous in that the length of the far edge must align with the length of the close edge of the next step and vice versa.

Groups arrive at the staircase according to a Poisson Arrival Process. When a group arrives, the size of the group is sampled from a distribution and each of the members of the group are introduced into the model. 

The initial locations of the agents within the staircase are found by first introducing them in a horizonal arrangement (i.e. shoulder-to-shoulder) in an open space before the staircase subject to the same attracion and repulsions which govern movement within the staircase. This open space then narrows to match the dimensions of the fist stair, serving as a funnel. This enables the groups to form an inital arrangement on the staircase which resembles a natural group walking formation.

Attractive Forces:
- Desire to reach intended destination atop the staircase by following a linear trajectory (most efficient route between points a and b is a straight line)
- Desire to maintain intended velocity
- Desire to maintain a certain distance between group members
- Desire to obey agreed upon social convention (walking on the right side vs left side)

Repulsive Forces:
- Desire not to get too close to other people 
    - the strength of this desire depends on whether the other agent is a member of one's group and the direction in which they are traversing (e.g. One would have a stronger desire to avoid contact with someone going in the opposite direction)

These attractive and repulsive forces are then combined into a utility function which determines where the agent will step on the next stair in the $x$ dimension (along the width of the stair). 

The final output of this model is a dataset of simulated steps on each stair and their associated agent. These steps can easily be converted to represent the cumulative pressure placed on each stair over the time interval in which the simulation is run. With a long enough sample, we can then extrapolate to a larger time interval by assuming periodicity in the stepping patterns. 


In [ ]:
# Imports

